In [1]:
import argparse
import copy
import numpy as np
import os
# import psutil
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#tf.enable_eager_execution()

from time import time
try:
    from tensorflow.python.ops.nn_ops import leaky_relu
except ImportError:
    from tensorflow.python.framework import ops
    from tensorflow.python.ops import math_ops
    
    def leaky_relu(features, alpha=0.2, name=None):
        with ops.name_scope(name, "LeakyRelu", [features, alpha]):
            features = ops.convert_to_tensor(features, name="features")
            alpha = ops.convert_to_tensor(alpha, name="alpha")
            return math_ops.maximum(alpha * features, features)
        

2023-05-18 12:18:38.298361: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/parth/hadoop-3.2.4/lib/native:
2023-05-18 12:18:38.298451: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from load_data import load_EOD_data, load_relation_data
from evaluator import evaluate

In [3]:
seed = 590
np.random.seed(seed)
tf.set_random_seed(seed)

In [4]:
class ReRaLSTM:
    def __init__(self, data_path, market_name, tickers_fname, relation_name,
                 emb_fname, parameters, steps=1, epochs=100, batch_size=None, flat=False, gpu=False, in_pro=False):

        
        seed = 590
        random.seed(seed)
        np.random.seed(seed)
        tf.set_random_seed(seed)
        
        self.data_path = data_path
        self.market_name = market_name
        self.tickers_fname = tickers_fname
        self.relation_name = relation_name
        
        # load data
        self.tickers = np.genfromtxt(tickers_fname,
                                     dtype=str, delimiter='\t', skip_header=False)

        print('tickers selected:', len(self.tickers))
        
        self.eod_data, self.mask_data, self.gt_data, self.price_data = load_EOD_data(data_path, market_name, self.tickers, steps)
        
        print(self.eod_data.shape)
        print(self.mask_data.shape)
        print(self.gt_data.shape)
        print(self.price_data.shape)
        print('########')
        
        # relation data
        rname_tail = {'sector_industry': '_industry_relation.npy',
                      'wikidata': '_wiki_relation.npy'}

        self.rel_encoding, self.rel_mask = load_relation_data(
            os.path.join('relation', self.relation_name,
                         self.market_name + rname_tail[self.relation_name])
        )
        print('relation encoding shape:', self.rel_encoding.shape)
        print('relation mask shape:', self.rel_mask.shape)
        print('########')
        
        self.embedding = np.load(
            os.path.join('pretrain', emb_fname))
        print('embedding shape:', self.embedding.shape)

        self.parameters = copy.copy(parameters)
        self.steps = steps
        self.epochs = epochs
        self.flat = flat
        self.inner_prod = in_pro
        if batch_size is None:
            self.batch_size = len(self.tickers)
        else:
            self.batch_size = batch_size

        self.valid_index = 756
        self.test_index = 1008
        self.trade_dates = self.mask_data.shape[1]
        self.fea_dim = 5

        self.gpu = gpu
        print('########')
    
    def get_batch(self, offset=None):
        
        if offset is None:
            offset = random.randrange(0, self.valid_index)
            
        #print(offset)
        
        seq_len = self.parameters['seq']
        mask_batch = self.mask_data[:, offset: offset + seq_len + self.steps]
        
        #print('mask_batch dim1 = ', mask_batch.shape)
        mask_batch = np.min(mask_batch, axis=1)
        #print('mask_batch dim2 = ', mask_batch.shape)
        
        return self.embedding[:, offset, :], \
               np.expand_dims(mask_batch, axis=1), \
               np.expand_dims(
                   self.price_data[:, offset + seq_len - 1], axis=1
               ), \
               np.expand_dims(
                   self.gt_data[:, offset + seq_len + self.steps - 1], axis=1
               )
        
    def train(self):
        if self.gpu == True:
            device_name = '/gpu:0'
        else:
            device_name = '/cpu:0'
        print('device name:', device_name)
        
        with tf.device(device_name):
            tf.compat.v1.reset_default_graph()

            seed = 123456789
            random.seed(seed)
            np.random.seed(seed)
            tf.set_random_seed(seed)
            
            ground_truth = tf.placeholder(tf.float32, [self.batch_size, 1])
            mask = tf.placeholder(tf.float32, [self.batch_size, 1])
            feature = tf.placeholder(tf.float32,
                                     [self.batch_size, self.parameters['unit']])
            base_price = tf.placeholder(tf.float32, [self.batch_size, 1])
            all_one = tf.ones([self.batch_size, 1], dtype=tf.float32)

            relation = tf.constant(self.rel_encoding, dtype=tf.float32)
            rel_mask = tf.constant(self.rel_mask, dtype=tf.float32)

            rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
            
            print('#################################')
            print('ground_truth = ', ground_truth.shape)
            print('mask = ', mask.shape)
            print('feature = ', feature.shape)
            print('base_price = ', base_price.shape)
            print('all_one = ', all_one.shape)
            print('relation = ', relation.shape)
            print('rel_mask = ', rel_mask.shape)
            print('rel_weight = ', rel_weight.shape)
            print('#################################')

            if self.inner_prod:
                print('inner product weight')
                inner_weight = tf.matmul(feature, feature, transpose_b=True)
                print('inner_weight = ', inner_weight.shape)
                #print('inner_weight = ', inner_weight.numpy())
                #a = tf.Print(inner_weight, [inner_weight], message="This is a: ")
                #b = tf.add(a, a)
                #inner_weight.print()
                
                
                weight = tf.multiply(inner_weight, rel_weight[:, :, -1])
                print('weight = ', weight.shape)
            else:
                print('#################################')
                print('sum weight')
                head_weight = tf.layers.dense(feature, units=1,
                                              activation=leaky_relu)
                tail_weight = tf.layers.dense(feature, units=1,
                                              activation=leaky_relu)
                print('head_weight = ', head_weight.shape)
                print('tail_weight = ', tail_weight.shape)
                weight = tf.add(
                    tf.add(
                        tf.matmul(head_weight, all_one, transpose_b=True),
                        tf.matmul(all_one, tail_weight, transpose_b=True)
                    ), rel_weight[:, :, -1]
                )
                print('weight = ',weight.shape)
                print('#################################')
            
            
            weight_masked = tf.nn.softmax(tf.add(rel_mask, weight), dim=0)
            outputs_proped = tf.matmul(weight_masked, feature)

            print('weight_masked = ', weight_masked.shape)
            print('outputs_proped = ', outputs_proped.shape)
            
            if self.flat:
                print('one more hidden layer')
                outputs_concated = tf.layers.dense(
                    tf.concat([feature, outputs_proped], axis=1),
                    units=self.parameters['unit'], activation=leaky_relu,
                    kernel_initializer=tf.glorot_uniform_initializer()
                )
            else:
                outputs_concated = tf.concat([feature, outputs_proped], axis=1)

            print('outputs_concated = ', outputs_concated.shape)
            
            prediction = tf.layers.dense(
                outputs_concated, units=1, activation=leaky_relu, name='reg_fc',
                kernel_initializer=tf.glorot_uniform_initializer()
            ) 
            print('prediction type = ', type(prediction))
            print('prediction = ',prediction.shape)
            print('#################################')
            
            return_ratio = tf.div(tf.subtract(prediction, base_price), base_price)
            print('return_ratio type = ',type(return_ratio))
            print('return_ratio = ', return_ratio.shape)
            print('#################################')
            
            reg_loss = tf.losses.mean_squared_error(
                ground_truth, return_ratio, weights=mask
            )
            
            pre_pw_dif = tf.subtract(
                tf.matmul(return_ratio, all_one, transpose_b=True),
                tf.matmul(all_one, return_ratio, transpose_b=True)
            )
            gt_pw_dif = tf.subtract(
                tf.matmul(all_one, ground_truth, transpose_b=True),
                tf.matmul(ground_truth, all_one, transpose_b=True)
            )
            print('reg_loss = ',reg_loss.shape)
            print('pre_pw_dif = ', pre_pw_dif.shape)
            print('gt_pw_dif = ', gt_pw_dif.shape)
            
            mask_pw = tf.matmul(mask, mask, transpose_b=True)
            rank_loss = tf.reduce_mean(
                tf.nn.relu(
                    tf.multiply(
                        tf.multiply(pre_pw_dif, gt_pw_dif),
                        mask_pw
                    )
                )
            )
            print('rank_loss = ', rank_loss.shape)
            loss = reg_loss + tf.cast(self.parameters['alpha'], tf.float32) * \
                              rank_loss
            
            optimizer = tf.train.AdamOptimizer(
                learning_rate=self.parameters['lr']
            ).minimize(loss)
            
            
            print('#################################')
            
        sess = tf.Session()
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        #b.eval(session=sess)
        #print(sess.run(inner_weight))
        
        #print('inner_weight = ', inner_weight.eval(session=sess))
        
        
        best_valid_pred = np.zeros(
            [len(self.tickers), self.test_index - self.valid_index],
            dtype=float
        )
        best_valid_gt = np.zeros(
            [len(self.tickers), self.test_index - self.valid_index],
            dtype=float
        )
        best_valid_mask = np.zeros(
            [len(self.tickers), self.test_index - self.valid_index],
            dtype=float
        )
        best_test_pred = np.zeros(
            [len(self.tickers), self.trade_dates - self.parameters['seq'] -
             self.test_index - self.steps + 1], dtype=float
        )
        best_test_gt = np.zeros(
            [len(self.tickers), self.trade_dates - self.parameters['seq'] -
             self.test_index - self.steps + 1], dtype=float
        )
        best_test_mask = np.zeros(
            [len(self.tickers), self.trade_dates - self.parameters['seq'] -
             self.test_index - self.steps + 1], dtype=float
        )
        best_valid_perf = {
            'mse': np.inf, 'mrrt': 0.0, 'btl': 0.0
        }
        best_test_perf = {
            'mse': np.inf, 'mrrt': 0.0, 'btl': 0.0
        }
        best_valid_loss = np.inf

        batch_offsets = np.arange(start=0, stop=self.valid_index, dtype=int)
        
        for i in range(self.epochs):
            print('Starting epoch = ',i)
            t1 = time()
            np.random.shuffle(batch_offsets)
            tra_loss = 0.0
            tra_reg_loss = 0.0
            tra_rank_loss = 0.0
            
            for j in range(self.valid_index - self.parameters['seq'] -
                                   self.steps + 1):
                emb_batch, mask_batch, price_batch, gt_batch = self.get_batch(
                    batch_offsets[j])
                feed_dict = {
                    feature: emb_batch,
                    mask: mask_batch,
                    ground_truth: gt_batch,
                    base_price: price_batch
                }
                
                
                
                cur_loss, cur_reg_loss, cur_rank_loss, batch_out = \
                    sess.run((loss, reg_loss, rank_loss, optimizer),
                             feed_dict)
                #inner_weight.eval(session=tf.Session())
                
                tra_loss += cur_loss
                tra_reg_loss += cur_reg_loss
                tra_rank_loss += cur_rank_loss
                
            print('Train Loss:',
                  tra_loss / (self.valid_index - self.parameters['seq'] - self.steps + 1),
                  tra_reg_loss / (self.valid_index - self.parameters['seq'] - self.steps + 1),
                  tra_rank_loss / (self.valid_index - self.parameters['seq'] - self.steps + 1))
            
            # test on validation set
            cur_valid_pred = np.zeros(
                [len(self.tickers), self.test_index - self.valid_index],
                dtype=float
            )
            cur_valid_gt = np.zeros(
                [len(self.tickers), self.test_index - self.valid_index],
                dtype=float
            )
            cur_valid_mask = np.zeros(
                [len(self.tickers), self.test_index - self.valid_index],
                dtype=float
            )
            val_loss = 0.0
            val_reg_loss = 0.0
            val_rank_loss = 0.0
            
            for cur_offset in range(
                self.valid_index - self.parameters['seq'] - self.steps + 1,
                self.test_index - self.parameters['seq'] - self.steps + 1):
                
                emb_batch, mask_batch, price_batch, gt_batch = self.get_batch(
                    cur_offset)
                feed_dict = {
                    feature: emb_batch,
                    mask: mask_batch,
                    ground_truth: gt_batch,
                    base_price: price_batch
                }
                cur_loss, cur_reg_loss, cur_rank_loss, cur_rr, = \
                    sess.run((loss, reg_loss, rank_loss,
                              return_ratio), feed_dict)
                
                val_loss += cur_loss
                val_reg_loss += cur_reg_loss
                val_rank_loss += cur_rank_loss
                cur_valid_pred[:, cur_offset - (self.valid_index -
                                                self.parameters['seq'] -
                                                self.steps + 1)] = \
                    copy.copy(cur_rr[:, 0])
                cur_valid_gt[:, cur_offset - (self.valid_index -
                                              self.parameters['seq'] -
                                              self.steps + 1)] = \
                cur_valid_mask[:, cur_offset - (self.valid_index -
                                                self.parameters['seq'] -
                                                self.steps + 1)] = \
                    copy.copy(mask_batch[:, 0])
            print('Valid MSE:',
                  val_loss / (self.test_index - self.valid_index),
                  val_reg_loss / (self.test_index - self.valid_index),
                  val_rank_loss / (self.test_index - self.valid_index))
            cur_valid_perf = evaluate(cur_valid_pred, cur_valid_gt,
                                      cur_valid_mask)
            print('\t Valid preformance:', cur_valid_perf)
            
            #test on testing set
            cur_test_pred = np.zeros(
                [len(self.tickers), self.trade_dates - self.test_index],
                dtype=float
            )
            cur_test_gt = np.zeros(
                [len(self.tickers), self.trade_dates - self.test_index],
                dtype=float
            )
            cur_test_mask = np.zeros(
                [len(self.tickers), self.trade_dates - self.test_index],
                dtype=float
            )
            test_loss = 0.0
            test_reg_loss = 0.0
            test_rank_loss = 0.0
            
            for cur_offset in range(
                                            self.test_index - self.parameters['seq'] - self.steps + 1,
                                            self.trade_dates - self.parameters['seq'] - self.steps + 1
            ):
                emb_batch, mask_batch, price_batch, gt_batch = self.get_batch(
                    cur_offset)
                feed_dict = {
                    feature: emb_batch,
                    mask: mask_batch,
                    ground_truth: gt_batch,
                    base_price: price_batch
                }
                cur_loss, cur_reg_loss, cur_rank_loss, cur_rr = \
                    sess.run((loss, reg_loss, rank_loss,
                              return_ratio), feed_dict)
                test_loss += cur_loss
                test_reg_loss += cur_reg_loss
                test_rank_loss += cur_rank_loss

                cur_test_pred[:, cur_offset - (self.test_index -
                                               self.parameters['seq'] -
                                               self.steps + 1)] = \
                    copy.copy(cur_rr[:, 0])
                cur_test_gt[:, cur_offset - (self.test_index -
                                             self.parameters['seq'] -
                                             self.steps + 1)] = \
                    copy.copy(gt_batch[:, 0])
                cur_test_mask[:, cur_offset - (self.test_index -
                                               self.parameters['seq'] -
                                               self.steps + 1)] = \
                    copy.copy(mask_batch[:, 0])

            print('Test MSE:',
                  test_loss / (self.trade_dates - self.test_index),
                  test_reg_loss / (self.trade_dates - self.test_index),
                  test_rank_loss / (self.trade_dates - self.test_index))
            cur_test_perf = evaluate(cur_test_pred, cur_test_gt, cur_test_mask)
            print('\t Test performance:', cur_test_perf)
            
            if val_loss / (self.test_index - self.valid_index) < \
                    best_valid_loss:
                best_valid_loss = val_loss / (self.test_index -
                                              self.valid_index)
                best_valid_perf = copy.copy(cur_valid_perf)
                best_valid_gt = copy.copy(cur_valid_gt)
                best_valid_pred = copy.copy(cur_valid_pred)
                best_valid_mask = copy.copy(cur_valid_mask)
                best_test_perf = copy.copy(cur_test_perf)
                best_test_gt = copy.copy(cur_test_gt)
                best_test_pred = copy.copy(cur_test_pred)
                best_test_mask = copy.copy(cur_test_mask)
                print('Better valid loss:', best_valid_loss)
            t4 = time()
            print('epoch:', i, ('time: %.4f ' % (t4 - t1)))
        print('\nBest Valid performance:', best_valid_perf)
        print('\tBest Test performance:', best_test_perf)
        sess.close()
        tf.reset_default_graph()
        return best_valid_pred, best_valid_gt, best_valid_mask, \
               best_test_pred, best_test_gt, best_test_mask

        

In [5]:
if __name__ == '__main__':
    
    parameters = {'seq': int(16), 'unit': int(64), 'lr': float(0.001),
                  'alpha': float(1)}
    
    
    RR_LSTM = ReRaLSTM(
        data_path='2013-01-01',
        market_name='NASDAQ',
        tickers_fname='NASDAQ_tickers_qualify_dr-0.98_min-5_smooth.csv',
        relation_name='wikidata',
        emb_fname='NASDAQ_5_val.csv.npy',
        parameters=parameters,
        steps=1, epochs=50, batch_size=None, gpu=False,
        in_pro=False
    )

tickers selected: 1026
single EOD data shape: (1245, 6)
(1026, 1245, 5)
(1026, 1245)
(1026, 1245)
(1026, 1245)
########
relation encoding shape: (1026, 1026, 43)
relation encoding shape: (1026, 1026, 43)
relation mask shape: (1026, 1026)
########
embedding shape: (1026, 1245, 64)
########


In [6]:
#alpha=1, itr = 1, rel = wiki, NASDAQ, implicit, nasdaq_5_val, alpha = 1
pred_all = RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 43)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
#################################
sum weight
head_weight =  (1026, 1)
tail_weight =  (1026, 1)
weight =  (1026, 1026)
#################################
Instructions for updating:
dim is deprecated, use axis instead
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/tmp/ipykernel_4033/4199189488.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_4033/4199189488.py:140: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  head_weight = tf.layers.dense(feature, units=1,
/tmp/ipykernel_4033/4199189488.py:142: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tail_weight = tf.layers.dense(feature, units=1,
/tmp/ipykernel_4033/4199189488.py:174: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################


2023-05-18 12:19:16.063620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/parth/hadoop-3.2.4/lib/native:
2023-05-18 12:19:16.063646: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-18 12:19:16.063686: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parth-HP-ProBook-430-G6): /proc/driver/nvidia/version does not exist
2023-05-18 12:19:16.065177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 12:19:16.386034: I tensorflow/com

Starting epoch =  0
Train Loss: 0.05139636800095842 0.04963764668240942 0.0017587215498064618
Valid MSE: 0.01593795862965404 0.014780236940298761 0.0011577217501037502
	 Valid preformance: {'mse': 1.0028498349228572, 'mrrt': 0.006610101120312123, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.022400403045557973 0.02129613506344426 0.0011042680587415202
	 Test performance: {'mse': 0.021301633176754466, 'mrrt': 0.02303056381475453, 'btl': 1.2296989791211672, 'btl5': 1.0422350243257823, 'btl10': 1.1532495740141717}
Better valid loss: 0.01593795862965404
epoch: 0 time: 105.7131 
Starting epoch =  1
Train Loss: 0.01356556700366373 0.012570421030191152 0.0009951459803945713
Valid MSE: 0.011471633233928255 0.010492988084147255 0.00097864516479488
	 Valid preformance: {'mse': 1.0290695323401053, 'mrrt': 0.004457130327837436, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.019518808300610837 0.018547326346018394 0.0009714819825910629
	 Test performance: {'mse': 0.018553156797

Valid MSE: 0.01017794497717645 0.009256424363850364 0.0009215205897658413
	 Valid preformance: {'mse': 1.0165425010947458, 'mrrt': 0.0076572522155907005, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.016682819518327462 0.015778241659424225 0.000904577823536557
	 Test performance: {'mse': 0.015783136761694034, 'mrrt': 0.030643078294203912, 'btl': 1.5187746252631769, 'btl5': 1.2611276438197825, 'btl10': 1.2406962269883488}
epoch: 14 time: 83.9645 
Starting epoch =  15
Train Loss: 0.010479415844295275 0.009610051970702369 0.000869363866356277
Valid MSE: 0.010424506606622821 0.00950421923993244 0.000920287357451069
	 Valid preformance: {'mse': 0.9919451504154091, 'mrrt': 0.011233644049001337, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.01649341821073228 0.015555018040386937 0.0009384001428379288
	 Test performance: {'mse': 0.015560217854058117, 'mrrt': 0.03258639209024591, 'btl': 0.8868498670344707, 'btl5': 1.2889644297451017, 'btl10': 1.1805359098420019}
epoch: 15 

Train Loss: 0.010317645808430137 0.009454872906585601 0.0008627729056988264
Valid MSE: 0.010191561935085153 0.00929579265703935 0.0008957692742345869
	 Valid preformance: {'mse': 1.0499757909330827, 'mrrt': 0.011024495222125072, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.019201299934038886 0.01831599447153163 0.000885305466928091
	 Test performance: {'mse': 0.018322967522214444, 'mrrt': 0.02688785699886191, 'btl': 1.1910599311522674, 'btl5': 1.452608525546387, 'btl10': 1.3353856638433172}
epoch: 29 time: 73.0371 
Starting epoch =  30
Train Loss: 0.010370844219987458 0.009506071682364958 0.0008647725458030362
Valid MSE: 0.01010876833756883 0.009209380222720996 0.0008993880933664354
	 Valid preformance: {'mse': 1.0409398554888536, 'mrrt': 0.009203789172485252, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.018621164346546062 0.0177273843908989 0.0008937799804529592
	 Test performance: {'mse': 0.017733753040175895, 'mrrt': 0.025959916216963985, 'btl': 0.39913873354

Train Loss: 0.010356459584766747 0.009493186041662419 0.0008632735321313973
Valid MSE: 0.01058620366910916 0.009643565190749036 0.0009426384852896075
	 Valid preformance: {'mse': 0.9906991548855714, 'mrrt': 0.007286138386178747, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.015310743293147298 0.014412611019542197 0.0008981322733594993
	 Test performance: {'mse': 0.014417624937092376, 'mrrt': 0.03267286877549462, 'btl': 1.5454066387028433, 'btl5': 1.4470948419795626, 'btl10': 1.2771206032654838}
epoch: 44 time: 77.3279 
Starting epoch =  45
Train Loss: 0.010323023468230826 0.009459288554215753 0.0008637348978899772
Valid MSE: 0.010070416516446997 0.009168012368877137 0.0009024041222772443
	 Valid preformance: {'mse': 1.0113889059122654, 'mrrt': 0.007311065596273321, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.017561808491778023 0.01663224562955431 0.0009295628646797332
	 Test performance: {'mse': 0.016638219935421836, 'mrrt': 0.02823838954036893, 'btl': 0.7962022

In [10]:
#alpha=1, itr = 1, rel = wiki, NASDAQ, explicit, nasdaq_5_val, alpha = 0.1
pred_all = RR_LSTM.train()

device name: /cpu:0


/tmp/ipykernel_3826/4199189488.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_3826/4199189488.py:174: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 43)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.06198734935826144 0.06180164578899339 0.001857036366945485
Valid MSE: 0.014869066823776516 0.01475233249440198 0.001167343106187348
	 Valid preformance: {'mse': 1.01

Train Loss: 0.010000205094724691 0.009912086514177153 0.0008811858113748147
Valid MSE: 0.009945762740995085 0.009852200094240881 0.0009356266241483686
	 Valid preformance: {'mse': 0.9809203453717593, 'mrrt': 0.009704941693564536, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.015044502576003599 0.014950033689884446 0.0009446885637675691
	 Test performance: {'mse': 0.014954340573849206, 'mrrt': 0.04162138544032114, 'btl': 2.03626690278179, 'btl5': 1.428446331538727, 'btl10': 1.285118021371726}
epoch: 13 time: 2145.2259 
Starting epoch =  14
Train Loss: 0.009919862219484874 0.009831953845993691 0.0008790837500143695
Valid MSE: 0.009498765536894401 0.009405997788740529 0.0009276771704049131
	 Valid preformance: {'mse': 1.0144327901591277, 'mrrt': 0.011937307176743179, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.01603685757577545 0.015945363294663308 0.0009149428862756127
	 Test performance: {'mse': 0.015950220249610982, 'mrrt': 0.04450050295006438, 'btl': 2.02092690

Train Loss: 0.010032189033321433 0.009943934923944039 0.0008825412268256391
Valid MSE: 0.009613290870754373 0.009520565932025276 0.0009272492142848759
	 Valid preformance: {'mse': 0.9967330254764756, 'mrrt': 0.011262486419506315, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.01564782501084392 0.01555431101722682 0.0009351402385069945
	 Test performance: {'mse': 0.015559019641687809, 'mrrt': 0.03417951875604185, 'btl': 1.9944612335821148, 'btl5': 1.5837734935164927, 'btl10': 1.3664978180342586}
epoch: 28 time: 79.1493 
Starting epoch =  29
Train Loss: 0.00990162688131268 0.00981378155961834 0.0008784533780763505
Valid MSE: 0.00955860935232883 0.009467539553057461 0.0009106978856531592
	 Valid preformance: {'mse': 1.0468877657939024, 'mrrt': 0.012496993593100832, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.01818836780602922 0.01809823189622365 0.0009013592628545996
	 Test performance: {'mse': 0.018104075566114906, 'mrrt': 0.03582167768301086, 'btl': 1.811304261529

Train Loss: 0.010035474923112103 0.009947234679114174 0.0008824024776962737
Valid MSE: 0.009805198095458012 0.009710808735459096 0.0009438939152806554
	 Valid preformance: {'mse': 1.00771284348497, 'mrrt': 0.014796812796986804, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.016011162995034633 0.01591905983329951 0.0009210316031063094
	 Test performance: {'mse': 0.01592404191931252, 'mrrt': 0.04004563403693722, 'btl': 1.3043108869169373, 'btl5': 1.538013114643399, 'btl10': 1.3430738583144382}
epoch: 43 time: 75.5191 
Starting epoch =  44
Train Loss: 0.009974507889332804 0.009886461810989155 0.0008804606881411117
Valid MSE: 0.010232398966475138 0.010135441488172445 0.0009695746437443183
	 Valid preformance: {'mse': 0.9814326502140184, 'mrrt': 0.014299047095299415, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.01418312686153605 0.014091417409542492 0.000917094656735863
	 Test performance: {'mse': 0.014095217400356623, 'mrrt': 0.03394708237262807, 'btl': 1.436974901240

In [8]:
#alpha=1, itr = 1, rel = wiki, NASDAQ, explicit, nasdaq_5_val, alpha = 10
pred_all = RR_LSTM.train()

device name: /cpu:0


/tmp/ipykernel_3826/4199189488.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_3826/4199189488.py:174: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 43)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.07641368025309733 0.05864430768792895 0.0017769372633444397
Valid MSE: 0.024012849278127153 0.013108799721868266 0.0010904049635255
	 Valid preformance: {'mse': 1.02

Train Loss: 0.01872387496326622 0.01000637564858472 0.000871749930996196
Valid MSE: 0.019243023129150508 0.00986079941873276 0.0009382223678773106
	 Valid preformance: {'mse': 0.9896202324327532, 'mrrt': 0.009663749193709908, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.024894956051360204 0.015475665879306159 0.0009419290161738761
	 Test performance: {'mse': 0.015480444970092261, 'mrrt': 0.03782446649928871, 'btl': 1.3686649917799514, 'btl5': 1.3415272470068889, 'btl10': 1.3811765202942288}
epoch: 13 time: 71.4355 
Starting epoch =  14
Train Loss: 0.018653624277008143 0.009933773425055316 0.0008719850901822635
Valid MSE: 0.018566384889362825 0.0093627148548082 0.0009203670040098211
	 Valid preformance: {'mse': 1.0159167175208377, 'mrrt': 0.00653528711122028, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.02552532968956207 0.016315065595451036 0.0009210264128004102
	 Test performance: {'mse': 0.016320044056145327, 'mrrt': 0.03845464705591296, 'btl': 1.5345038737577

Train Loss: 0.018696450146317885 0.009971724994908514 0.0008724725149600212
Valid MSE: 0.0185536491048951 0.009399173389528953 0.000915447566570485
	 Valid preformance: {'mse': 1.0092344543314014, 'mrrt': 0.00741779452733714, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.026052148635440236 0.0166789764648437 0.0009373172138668282
	 Test performance: {'mse': 0.016684340064663315, 'mrrt': 0.03753058994498286, 'btl': 1.8749602499592584, 'btl5': 1.417823812476127, 'btl10': 1.3663342785987878}
epoch: 28 time: 74.2142 
Starting epoch =  29
Train Loss: 0.01859236008840034 0.00989454419578652 0.0008697815906851099
Valid MSE: 0.018559024432703616 0.009569050712762253 0.0008989973775608214
	 Valid preformance: {'mse': 1.0509585131422645, 'mrrt': 0.008014275359752876, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.02851739082532593 0.019259915274532535 0.0009257475531636442
	 Test performance: {'mse': 0.019265941573883973, 'mrrt': 0.030981907540513748, 'btl': 1.26075446599861

Train Loss: 0.01876239590976085 0.010022849911144254 0.0008739545995627555
Valid MSE: 0.01873438089849457 0.009530715958496172 0.0009203664903041153
	 Valid preformance: {'mse': 1.0273246819592121, 'mrrt': 0.00886497580552067, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.026870500154887574 0.017613757822175318 0.0009256742355798839
	 Test performance: {'mse': 0.017619470151622026, 'mrrt': 0.03568301294347037, 'btl': 1.2643818592478056, 'btl5': 1.5329182681191003, 'btl10': 1.2585899957349278}
epoch: 43 time: 73.3538 
Starting epoch =  44
Train Loss: 0.01866692148425893 0.00995034078211599 0.0008716580724246121
Valid MSE: 0.019131551141895 0.009658068137627745 0.0009473482999647598
	 Valid preformance: {'mse': 1.0034888514038365, 'mrrt': 0.008661682023565298, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.024477964369305074 0.015333744689413516 0.0009144219748660105
	 Test performance: {'mse': 0.01533826199133995, 'mrrt': 0.032213326472670445, 'btl': 1.1162541551748

In [6]:
#alpha=1, itr = 1, rel = wiki, NASDAQ, explicit, nasdaq_5_val, alpha = 1
pred_all = RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 43)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
Instructions for updating:
dim is deprecated, use axis instead
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/tmp/ipykernel_3826/4199189488.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_3826/4199189488.py:174: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################


2023-05-17 11:31:10.558471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/parth/hadoop-3.2.4/lib/native:
2023-05-17 11:31:10.558501: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-17 11:31:10.558544: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parth-HP-ProBook-430-G6): /proc/driver/nvidia/version does not exist
2023-05-17 11:31:10.559423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 11:31:10.822897: I tensorflow/com

Starting epoch =  0
Train Loss: 0.06332267386889136 0.06147366883233189 0.0018490050928793042
Valid MSE: 0.015713433908771664 0.01455489921915744 0.0011585347237996757
	 Valid preformance: {'mse': 1.0161046383482348, 'mrrt': 0.008595117304865963, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.02306013627032816 0.021941667281185524 0.0011184689424782678
	 Test performance: {'mse': 0.021947036186065964, 'mrrt': 0.02424255959046431, 'btl': 0.7308219000115059, 'btl5': 1.090960850882948, 'btl10': 1.2552359142627212}
Better valid loss: 0.015713433908771664
epoch: 0 time: 92.2811 
Starting epoch =  1
Train Loss: 0.012634350221000007 0.011670216331510126 0.0009641338758032165
Valid MSE: 0.010767415698085512 0.009830452895368495 0.0009369628163450028
	 Valid preformance: {'mse': 1.0294046609505485, 'mrrt': 0.005526684630266208, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.019311673868494698 0.018355312827927402 0.0009563610253399749
	 Test performance: {'mse': 0.0183610221

Valid MSE: 0.01032214571115753 0.009395462207658778 0.0009266834739329559
	 Valid preformance: {'mse': 1.0154711824600713, 'mrrt': 0.009841372431431943, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.016919669608746652 0.0160052421098017 0.0009144274805248067
	 Test performance: {'mse': 0.01601011555236361, 'mrrt': 0.04033352042262374, 'btl': 1.950877166207647, 'btl5': 1.4914795456686991, 'btl10': 1.3603989983002078}
epoch: 14 time: 75.5872 
Starting epoch =  15
Train Loss: 0.010788992832641344 0.009909211950552827 0.0008797808814985753
Valid MSE: 0.01046246939360918 0.009541607801876371 0.0009208615708288677
	 Valid preformance: {'mse': 0.9994603279961656, 'mrrt': 0.008920301573250201, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.017164712909310428 0.01621739016177785 0.0009473227612862859
	 Test performance: {'mse': 0.01622238981108089, 'mrrt': 0.03829868694235478, 'btl': 1.8919692334893625, 'btl5': 1.5239157464879112, 'btl10': 1.3904981484995922}
epoch: 15 time

Train Loss: 0.010688832842719716 0.009811617591972086 0.0008772152531860598
Valid MSE: 0.010371700163915872 0.009463384019447461 0.0009083161227560291
	 Valid preformance: {'mse': 1.047198666407881, 'mrrt': 0.0085041776380017, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.01912929871382844 0.01822463429301097 0.0009046644316239573
	 Test performance: {'mse': 0.01823048989026571, 'mrrt': 0.03671560504373343, 'btl': 1.7621323960775044, 'btl5': 1.5199443057703321, 'btl10': 1.4157564109220417}
epoch: 29 time: 77.8421 
Starting epoch =  30
Train Loss: 0.010789495982179369 0.00990838626449978 0.0008811097187021546
Valid MSE: 0.010440011037557963 0.009512253397602647 0.0009277576524283855
	 Valid preformance: {'mse': 1.028941740445581, 'mrrt': 0.009269702921194655, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.017983316990079257 0.017074796764125048 0.0009085202740921879
	 Test performance: {'mse': 0.01708016472579557, 'mrrt': 0.03651668243629997, 'btl': 1.67310516294674

Train Loss: 0.010766718435302578 0.00988735150847886 0.0008793669285542155
Valid MSE: 0.011053809824044861 0.010086267898302703 0.0009675419435278321
	 Valid preformance: {'mse': 0.9831827317853505, 'mrrt': 0.014418972636043111, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.015101830797262081 0.014184857526094602 0.0009169732264679255
	 Test performance: {'mse': 0.01418871476776008, 'mrrt': 0.03389615238852802, 'btl': 1.4063438127050176, 'btl5': 1.4158014660381013, 'btl10': 1.2331845769425869}
epoch: 44 time: 74.0945 
Starting epoch =  45
Train Loss: 0.010716914515187208 0.009837631740909372 0.0008792827715247718
Valid MSE: 0.01031435453998191 0.009400772912028645 0.0009135816511670349
	 Valid preformance: {'mse': 1.0141535465353289, 'mrrt': 0.012278599592526883, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.018012949851054682 0.017069841169198102 0.0009431086862774149
	 Test performance: {'mse': 0.01707528778417774, 'mrrt': 0.03498896771892538, 'btl': 1.793495047

In [10]:
#alpha=1, itr = 1, rel = sector, NASDAQ, explicit, nasdaq_5_val, alpha = 10
pred_all = RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
Instructions for updating:
dim is deprecated, use axis instead
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/tmp/ipykernel_3686/4199189488.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_3686/4199189488.py:174: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(
2023-05-16 12:06:02.136679: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/parth/hadoop-3.2.4/lib/native:
2023-05-16 12:06:02.136711: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-16 12:06:02.136735: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parth-HP-Pro

return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################


2023-05-16 12:06:02.589143: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Starting epoch =  0


2023-05-16 12:06:03.505783: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 408438288 exceeds 10% of free system memory.
2023-05-16 12:06:05.319375: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 408438288 exceeds 10% of free system memory.
2023-05-16 12:06:05.319416: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 408438288 exceeds 10% of free system memory.
2023-05-16 12:06:06.308235: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 408438288 exceeds 10% of free system memory.
2023-05-16 12:06:06.510001: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 408438288 exceeds 10% of free system memory.


Train Loss: 0.08683517980011733 0.06702139222037953 0.0019813787581468303
Valid MSE: 0.02940651820972562 0.01681893522156373 0.0012587583094875933
	 Valid preformance: {'mse': 1.0129782305324795, 'mrrt': 0.013600803079681293, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.03344011021471476 0.021639716587489165 0.001180039362869921
	 Test performance: {'mse': 0.021643230403288662, 'mrrt': 0.024468552929984518, 'btl': 0.8385713045136072, 'btl5': 1.1722208994964611, 'btl10': 1.0667503145974482}
Better valid loss: 0.02940651820972562
epoch: 0 time: 153.1539 
Starting epoch =  1
Train Loss: 0.023576915826698814 0.013302134521104195 0.0010274781294896996
Valid MSE: 0.020213535140519812 0.010535554601884788 0.0009677980475345744
	 Valid preformance: {'mse': 1.0401971213909014, 'mrrt': 0.00894795127367693, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.029532881752285273 0.019579131030457934 0.0009953750742949105
	 Test performance: {'mse': 0.019585175614801924, 'mrrt': 0.0

Train Loss: 0.016573576850665583 0.008467313212443244 0.0008106263650571802
Valid MSE: 0.017025810629425066 0.008323394007877342 0.0008702416597525515
	 Valid preformance: {'mse': 1.017793798134224, 'mrrt': 0.008313119042864185, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.02549936829079807 0.016746350198606902 0.0008753018175204627
	 Test performance: {'mse': 0.01675212246762005, 'mrrt': 0.026873536920319317, 'btl': 1.7965579799492843, 'btl5': 1.577501037273578, 'btl10': 1.3579274215818262}
epoch: 14 time: 110.5526 
Starting epoch =  15
Train Loss: 0.01662030397486445 0.008512406470254063 0.0008107897478652916
Valid MSE: 0.01711476539882521 0.008477278700512316 0.0008637486749822052
	 Valid preformance: {'mse': 0.9936420419867251, 'mrrt': 0.006173781204202959, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.025613616837605144 0.016491179154077662 0.0009122437638827926
	 Test performance: {'mse': 0.016496660963536056, 'mrrt': 0.026784267747895774, 'btl': 1.69979914

	 Test performance: {'mse': 0.017323129230287098, 'mrrt': 0.033008221140129676, 'btl': 1.752383919490967, 'btl5': 1.3986252416216303, 'btl10': 1.3156241450337807}
epoch: 28 time: 113.7266 
Starting epoch =  29
Train Loss: 0.01637993351480848 0.008350054118656428 0.0008029879411018604
Valid MSE: 0.016803216078274306 0.008342543166012519 0.0008460672900019702
	 Valid preformance: {'mse': 1.0400944012471276, 'mrrt': 0.005871900555180425, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.02821298518890067 0.019254039918005717 0.0008958945262544486
	 Test performance: {'mse': 0.019260390018702146, 'mrrt': 0.0285766009450143, 'btl': 1.671638734114822, 'btl5': 1.414630733779632, 'btl10': 1.3551544226788799}
epoch: 29 time: 113.7971 
Starting epoch =  30
Train Loss: 0.01645892924956373 0.008397632942383958 0.0008061296312843223
Valid MSE: 0.016903640894544502 0.008326215183524976 0.0008577425717718027
	 Valid preformance: {'mse': 1.0260180062183797, 'mrrt': 0.0066581238368211984, 'btl':

Test MSE: 0.027797526707440488 0.018732016194879002 0.0009065510489966107
	 Test performance: {'mse': 0.018738768804607023, 'mrrt': 0.025985228446330187, 'btl': 1.0817579319118522, 'btl5': 1.2785927296150483, 'btl10': 1.3153487527069052}
epoch: 43 time: 120.3590 
Starting epoch =  44
Train Loss: 0.016424117015825735 0.0083815535731815 0.0008042563418417263
Valid MSE: 0.016971553306257915 0.008322714757736004 0.0008648838593794536
	 Valid preformance: {'mse': 1.0240120923494291, 'mrrt': 0.006644997672650575, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.02828815739743317 0.01934093420436991 0.0008947223204852157
	 Test performance: {'mse': 0.01934874100530716, 'mrrt': 0.02218903811861527, 'btl': 1.1246442015399225, 'btl5': 1.3995837378664873, 'btl10': 1.2391871951574105}
epoch: 44 time: 118.6669 
Starting epoch =  45
Train Loss: 0.01643014603319603 0.008385234387792848 0.0008044911629514064
Valid MSE: 0.01682241176742883 0.008338895966372793 0.0008483515798977192
	 Valid pref

In [ ]:
#alpha=1, itr = 1, rel = sector, NASDAQ, explicit, nasdaq_mean
pred_all = RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
Instructions for updating:
dim is deprecated, use axis instead
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/tmp/ipykernel_19411/4199189488.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_19411/4199189488.py:174: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################


2023-05-09 18:31:12.314874: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/parth/hadoop-3.2.4/lib/native:
2023-05-09 18:31:12.314919: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-09 18:31:12.314946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parth-HP-ProBook-430-G6): /proc/driver/nvidia/version does not exist
2023-05-09 18:31:12.315824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 18:31:13.135922: I tensorflow/com

Starting epoch =  0
Train Loss: 0.15853375750467988 0.15729411565069412 0.0012396418227581308
Valid MSE: 0.029738394403091027 0.028636634756352693 0.0011017596825406634
	 Valid preformance: {'mse': 1.1695964091124236, 'mrrt': 0.004183961616292055, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.07450309931791785 0.07323256557815186 0.0012705337009608808
	 Test performance: {'mse': 0.07326551136797407, 'mrrt': 0.013187531328590301, 'btl': 0.18970057426486164, 'btl5': 0.7765246743685569, 'btl10': 0.9570663604768929}
Better valid loss: 0.029738394403091027
epoch: 0 time: 157.7416 
Starting epoch =  1
Train Loss: 0.025001908432239213 0.024011759955838725 0.0009901484913456984
Valid MSE: 0.012034849873522208 0.011183927961004278 0.0008509219081292579
	 Valid preformance: {'mse': 1.0610968289177343, 'mrrt': 0.004058567469691535, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.03110445268012049 0.030133159780081062 0.0009712928818648815
	 Test performance: {'mse': 0.03014702

In [56]:
RR_LSTM.get_batch()

555
mask_batch dim1 =  (1026, 5)
mask_batch dim2 =  (1026,)


(array([[-3.0125060e-05, -1.7685685e-04, -6.4636532e-07, ...,
          6.3038402e-04,  1.0626104e-05, -2.1483536e-05],
        [-2.8822884e-05, -1.7107392e-04,  8.1819217e-06, ...,
          7.2274439e-04,  9.1834509e-06, -1.8919716e-05],
        [-1.7671644e-05, -1.1009837e-04,  1.7487325e-04, ...,
          5.4492807e-04,  4.7322019e-06, -1.0544056e-05],
        ...,
        [-4.2977026e-05, -2.4030759e-04, -3.1097486e-04, ...,
          7.6915562e-04,  1.8649675e-05, -3.7233414e-05],
        [-4.4837383e-05, -2.4946945e-04, -3.3468500e-04, ...,
          7.4169273e-04,  2.0047362e-05, -4.0081653e-05],
        [-5.4373058e-06, -4.0081028e-05,  1.9934858e-04, ...,
          3.7582795e-04,  9.8576459e-07, -2.5601478e-06]], dtype=float32),
 array([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]], dtype=float32),
 array([[0.613122],
        [0.617323],
        [0.7218  ],
        ...,
        [0.526865],
        [0.518151],
        [0.966908]], dt

In [83]:
RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)


/tmp/ipykernel_50128/2877286019.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)


In [93]:
RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
#################################
sum weight
head_weight =  (1026, 1)
tail_weight =  (1026, 1)
weight =  (1026, 1026)
#################################
outputs_concated =  (1026, 128)
prediction =  (1026, 1)
#################################


/tmp/ipykernel_50128/2584083937.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_50128/2584083937.py:134: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  head_weight = tf.layers.dense(feature, units=1,
/tmp/ipykernel_50128/2584083937.py:136: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tail_weight = tf.layers.dense(feature, units=1,
/tmp/ipykernel_50128/2584083937.py:165: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


In [14]:
pred_all = RR_LSTM.train()
#alpha = 1

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.10608068416304131 0.10524141082848498 0.0008392736290310544
Valid MSE: 0.004364873286307094 0.003849422346637954 0.0005154509270805792
	 Valid pr

Test MSE: 0.0004633088513397325 0.0004253548572801706 3.795399309250424e-05
	 Test performance: {'mse': 0.00042524911018399907, 'mrrt': 0.022787745116612922, 'btl': 1.0153988432430197}
epoch: 14 time: 109.9481 
Starting epoch =  15
Train Loss: 0.00048708246145365964 0.00045745686153610663 2.962559967297263e-05
Valid MSE: 0.0005398628824286066 0.0005087461233040177 3.111676079199588e-05
	 Valid preformance: {'mse': 0.9926218166872647, 'mrrt': 0.006820190281870942, 'btl': 253.0}
Test MSE: 0.0004521610557552592 0.0004137766099645941 3.8384444255652966e-05
	 Test performance: {'mse': 0.00041369187854754043, 'mrrt': 0.020658669340219427, 'btl': 0.6967393907252699}
Better valid loss: 0.0005398628824286066
epoch: 15 time: 104.6600 
Starting epoch =  16
Train Loss: 0.0004922039270978358 0.00046466153350617815 2.754239358551245e-05
Valid MSE: 0.0005332876069241181 0.0005054261439779241 2.7861462686338373e-05
	 Valid preformance: {'mse': 1.003297715802389, 'mrrt': 0.0066229096946975674, 'btl': 2

Train Loss: 0.00045846917004186056 0.0004439457174551019 1.4523452612568665e-05
Valid MSE: 0.0005149825529005039 0.0005002458297346334 1.4736723456403451e-05
	 Valid preformance: {'mse': 1.0029112248021805, 'mrrt': 0.005334884664788119, 'btl': 253.0}
Test MSE: 0.00040927728812921525 0.0003912355907370419 1.80416969546949e-05
	 Test performance: {'mse': 0.00039116513012375136, 'mrrt': 0.025858871634852756, 'btl': 1.0365304075239692}
epoch: 31 time: 103.1338 
Starting epoch =  32
Train Loss: 0.00046050815219279834 0.0004461762027121212 1.4331949145761846e-05
Valid MSE: 0.0005497699996109106 0.0005353160280690876 1.4453972638991202e-05
	 Valid preformance: {'mse': 1.0110885524033981, 'mrrt': 0.005910779025512636, 'btl': 253.0}
Test MSE: 0.00044225257249619766 0.00042464984652099006 1.76027258140313e-05
	 Test performance: {'mse': 0.0004246047596113326, 'mrrt': 0.0264008261896617, 'btl': 1.105587910511531}
epoch: 32 time: 107.2801 
Starting epoch =  33
Train Loss: 0.00046815890557293124 0.

	 Test performance: {'mse': 0.0004606149856142965, 'mrrt': 0.031629909679625125, 'btl': 1.5560290497378446}
epoch: 47 time: 106.7100 
Starting epoch =  48
Train Loss: 0.0004524841529436762 0.0004424833470398585 1.0000805842058046e-05
Valid MSE: 0.0005272567648226248 0.000517853901588491 9.402863882870508e-06
	 Valid preformance: {'mse': 1.0083070365975415, 'mrrt': 0.006290658545732941, 'btl': 253.0}
Test MSE: 0.0004127696242462451 0.0004039606756583752 8.808948597463682e-06
	 Test performance: {'mse': 0.00040390569020904186, 'mrrt': 0.03777941155095809, 'btl': 1.3925193942850456}
epoch: 48 time: 107.6282 
Starting epoch =  49
Train Loss: 0.00045708678599849746 0.00044721059270700237 9.876193750236829e-06
Valid MSE: 0.0005282714442901557 0.0005170502447562906 1.1221199770261592e-05
	 Valid preformance: {'mse': 0.9886683008471651, 'mrrt': 0.0059495051770165645, 'btl': 253.0}
Test MSE: 0.0004137732015008044 0.0004029372083917182 1.0835993274100045e-05
	 Test performance: {'mse': 0.0004028

In [16]:
pred_all = RR_LSTM.train()
#alpha = 0.1

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.10548896170001024 0.10540441241857873 0.0008454917283858309
Valid MSE: 0.004010789944524211 0.003958435969368097 0.0005235397304262272
	 Valid pr

Test MSE: 0.0004590929532890259 0.00045391504212188285 5.1779100481191855e-05
	 Test performance: {'mse': 0.00045381756623445236, 'mrrt': 0.010434497485103932, 'btl': 0.2899959494243376}
epoch: 14 time: 106.6509 
Starting epoch =  15
Train Loss: 0.00046929137380494787 0.00046548768331303034 3.803690735022994e-05
Valid MSE: 0.0005180088181597446 0.000513784721064555 4.224096648382358e-05
	 Valid preformance: {'mse': 0.9933339709945069, 'mrrt': 0.005450017956919112, 'btl': 253.0}
Test MSE: 0.0004316336154310968 0.00042688291902282524 4.75069576049644e-05
	 Test performance: {'mse': 0.0004268116277675156, 'mrrt': 0.01426106334238307, 'btl': 0.7014377297309693}
Better valid loss: 0.0005180088181597446
epoch: 15 time: 106.8535 
Starting epoch =  16
Train Loss: 0.0004757895428574065 0.000472172937608431 3.616604629904622e-05
Valid MSE: 0.0005169774457398388 0.0005132167658187442 3.760681623870845e-05
	 Valid preformance: {'mse': 1.003430408820516, 'mrrt': 0.004659870876747716, 'btl': 253.0}


Valid MSE: 0.0005049533404720327 0.0005026358905149483 2.3174508142469605e-05
	 Valid preformance: {'mse': 1.003294179669654, 'mrrt': 0.0047722940723958, 'btl': 253.0}
Test MSE: 0.0004094139613748532 0.00040661923572340514 2.7947264247104075e-05
	 Test performance: {'mse': 0.00040656454156823643, 'mrrt': 0.022288112673180212, 'btl': 0.29682383648469113}
epoch: 31 time: 103.8483 
Starting epoch =  32
Train Loss: 0.00045226291467032926 0.0004499575126533887 2.3054015468300645e-05
Valid MSE: 0.0005434118028215524 0.0005410474925791653 2.364310745496556e-05
	 Valid preformance: {'mse': 1.0117743402343136, 'mrrt': 0.004609608971313092, 'btl': 253.0}
Test MSE: 0.0004572973920391943 0.0004546296825649154 2.667709917513696e-05
	 Test performance: {'mse': 0.00045461125596041665, 'mrrt': 0.024061487173997226, 'btl': 0.6271612044365611}
epoch: 32 time: 101.9022 
Starting epoch =  33
Train Loss: 0.00046140246240365494 0.0004591592976499374 2.2431653805314625e-05
Valid MSE: 0.0005252054619458726 0.

Train Loss: 0.0004467765478546278 0.0004449553234145561 1.8212246283002135e-05
Valid MSE: 0.0005245461545114051 0.0005224442315848715 2.101922385528659e-05
	 Valid preformance: {'mse': 1.0088426793192329, 'mrrt': 0.004900795621121855, 'btl': 253.0}
Test MSE: 0.0004341125381198802 0.00043198123710811576 2.1313003528604574e-05
	 Test performance: {'mse': 0.00043195439578792155, 'mrrt': 0.0329699450107717, 'btl': 1.0179015222820453}
epoch: 48 time: 105.0792 
Starting epoch =  49
Train Loss: 0.00045150545083743053 0.00044964234907269744 1.863100955098481e-05
Valid MSE: 0.0005206991005425586 0.0005182325525859755 2.466547298643e-05
	 Valid preformance: {'mse': 0.989037239345035, 'mrrt': 0.004526461133991229, 'btl': 253.0}
Test MSE: 0.00039775616942240783 0.0003957779834784748 1.978186336512366e-05
	 Test performance: {'mse': 0.00039566672351605683, 'mrrt': 0.04101799169362999, 'btl': 1.5947886617213953}
epoch: 49 time: 104.1424 

Best Valid performance: {'mse': 1.0003895242194676, 'mrrt': 0

In [18]:
pred_all = RR_LSTM.train()
#alpha = 10

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.11175287173356156 0.1039131644660154 0.000783970741795693
Valid MSE: 0.007264402815111218 0.002897374612364238 0.0004367028195124548
	 Valid pref

	 Valid preformance: {'mse': 0.9986187532106641, 'mrrt': 0.006283298725888906, 'btl': 253.0}
Test MSE: 0.0004982572018939074 0.0003809420372146272 1.1731516419958893e-05
	 Test performance: {'mse': 0.0003808481235149157, 'mrrt': 0.030717674284009056, 'btl': 1.025695012853248}
epoch: 14 time: 97.5141 
Starting epoch =  15
Train Loss: 0.0005382677610582874 0.000435660251241643 1.0260751009440896e-05
Valid MSE: 0.0005992989302155495 0.0005049641823995718 9.433474769507267e-06
	 Valid preformance: {'mse': 0.9915945979652383, 'mrrt': 0.007386130586099821, 'btl': 253.0}
Test MSE: 0.0004808133327301969 0.00039351302840577964 8.730030406730271e-06
	 Test performance: {'mse': 0.000393401273130829, 'mrrt': 0.03804966336156718, 'btl': 1.4932176804868504}
Better valid loss: 0.0005992989302155495
epoch: 15 time: 97.5878 
Starting epoch =  16
Train Loss: 0.0005371657927360109 0.00043915096515824236 9.801482740447288e-06
Valid MSE: 0.0006390381888541881 0.0004930176707155567 1.4602051832991406e-05
	 

Valid MSE: 0.0005826334381708875 0.0004949840214745593 8.764941795229715e-06
	 Valid preformance: {'mse': 1.0002268269078953, 'mrrt': 0.00829065798624425, 'btl': 253.0}
Test MSE: 0.0004562624655997986 0.00037932647320929177 7.693599329232646e-06
	 Test performance: {'mse': 0.0003792383086087419, 'mrrt': 0.022041940231140845, 'btl': 1.3755239092570264}
epoch: 31 time: 108.7174 
Starting epoch =  32
Train Loss: 0.0005077381346160798 0.0004348129122828706 7.292522310320704e-06
Valid MSE: 0.0005833438232897721 0.0005079882471435999 7.535557596752153e-06
	 Valid preformance: {'mse': 1.0059788967361987, 'mrrt': 0.00878773032684416, 'btl': 253.0}
Test MSE: 0.0004593256939809675 0.0003930412513844449 6.628444421212292e-06
	 Test performance: {'mse': 0.0003929749405989754, 'mrrt': 0.027083633145407824, 'btl': 2.0069672829122283}
epoch: 32 time: 108.0022 
Starting epoch =  33
Train Loss: 0.0005193226451779452 0.00044174195303443213 7.758069223446246e-06
Valid MSE: 0.0005979477158804897 0.0004933

	 Valid preformance: {'mse': 1.0029240241722381, 'mrrt': 0.008204696938925872, 'btl': 253.0}
Test MSE: 0.00043048003721856494 0.0003828145921567282 4.766544510788709e-06
	 Test performance: {'mse': 0.0003827387819772273, 'mrrt': 0.025158924854792236, 'btl': 2.2323753581149504}
Better valid loss: 0.0005536061981397252
epoch: 48 time: 109.1650 
Starting epoch =  49
Train Loss: 0.0005056840558953243 0.00043705391832879063 6.863013669083778e-06
Valid MSE: 0.0005624196527360399 0.0004954896584898787 6.692999461970358e-06
	 Valid preformance: {'mse': 0.9952380515809287, 'mrrt': 0.008041949905588423, 'btl': 253.0}
Test MSE: 0.0004437483647221283 0.00038302069946677517 6.072766511720204e-06
	 Test performance: {'mse': 0.0003829201664884206, 'mrrt': 0.01906028365193441, 'btl': 0.6821412388235331}
epoch: 49 time: 109.3820 

Best Valid performance: {'mse': 1.0029240241722381, 'mrrt': 0.008204696938925872, 'btl': 253.0}
	Best Test performance: {'mse': 0.0003827387819772273, 'mrrt': 0.0251589248547

In [7]:
#alpha=1, itr = 2, rel = sector
pred_all = RR_LSTM.train()

device name: /cpu:0
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
Instructions for updating:
dim is deprecated, use axis instead
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif 

Test MSE: 0.0005105323843311181 0.00045370193104257706 5.683045276663687e-05
	 Test performance: {'mse': 0.0004536651767050776, 'mrrt': 0.0144442129089728, 'btl': 0.7288201435294468}
epoch: 12 time: 107.1035 
Starting epoch =  13
Train Loss: 0.0004935354564450618 0.0004597187058558071 3.381675083014793e-05
Valid MSE: 0.0005428635360283171 0.0005076833910448292 3.518014527221639e-05
	 Valid preformance: {'mse': 0.9945952568225658, 'mrrt': 0.005481641378088061, 'btl': 253.0}
Test MSE: 0.00047541737898089826 0.00042725793747398376 4.815944333357898e-05
	 Test performance: {'mse': 0.0004271793089048966, 'mrrt': 0.01369986915457887, 'btl': 0.4058374743035529}
Better valid loss: 0.0005428635360283171
epoch: 13 time: 106.4944 
Starting epoch =  14
Train Loss: 0.0004886591086687986 0.00045739908305836866 3.126002600372492e-05
Valid MSE: 0.0005632356117111778 0.0005287564184520364 3.447919422638185e-05
	 Valid preformance: {'mse': 0.9880560063040158, 'mrrt': 0.00501827917560997, 'btl': 253.0}
T

Train Loss: 0.0004677292781707365 0.000452400507001093 1.532877157707254e-05
Valid MSE: 0.0005228017064851768 0.0004969774635687899 2.5824241714554587e-05
	 Valid preformance: {'mse': 0.9994324103827263, 'mrrt': 0.0058737204908248485, 'btl': 253.0}
Test MSE: 0.0004063559201681586 0.00038623764132491396 2.0118279211647526e-05
	 Test performance: {'mse': 0.0003861704990272732, 'mrrt': 0.0420903067714656, 'btl': 1.2318229801603593}
epoch: 29 time: 109.2739 
Starting epoch =  30
Train Loss: 0.0004601120980845047 0.00044543688880465293 1.467520925895794e-05
Valid MSE: 0.0005135420038958915 0.0004989887075059833 1.4553295151984905e-05
	 Valid preformance: {'mse': 1.0026984644156571, 'mrrt': 0.005848540171408574, 'btl': 253.0}
Test MSE: 0.0004072225802329483 0.00039033122401967456 1.689135449022258e-05
	 Test performance: {'mse': 0.0003902645480552022, 'mrrt': 0.03145926871913679, 'btl': 1.2032125126570463}
epoch: 30 time: 109.6728 
Starting epoch =  31
Train Loss: 0.00045846917004186056 0.00

	 Test performance: {'mse': 0.0003921985053979654, 'mrrt': 0.04933159857855904, 'btl': 1.7400432677241042}
epoch: 45 time: 107.8756 
Starting epoch =  46
Train Loss: 0.00045563101216307465 0.00044501536213902986 1.0615649963821507e-05
Valid MSE: 0.0005111669259291837 0.0005001926678925797 1.0974258346987116e-05
	 Valid preformance: {'mse': 0.9931869897023369, 'mrrt': 0.0061296136930663, 'btl': 253.0}
Test MSE: 0.0003984516680796674 0.0003878266122884742 1.0625054614990138e-05
	 Test performance: {'mse': 0.0003877211431211362, 'mrrt': 0.03984024151144697, 'btl': 1.9603446893161163}
epoch: 46 time: 107.0496 
Starting epoch =  47
Train Loss: 0.00045474130198960443 0.000444847090369188 9.89421101480361e-06
Valid MSE: 0.0005824563721987596 0.0005717346136248486 1.0721759454532903e-05
	 Valid preformance: {'mse': 1.0161009990902476, 'mrrt': 0.006956596079357165, 'btl': 253.0}
Test MSE: 0.00047081986855801965 0.0004606403974459589 1.0179471655071292e-05
	 Test performance: {'mse': 0.000460614

In [10]:
#alpha=1, itr = 3, rel = sector
pred_all = RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.10608068416304131 0.10524141082848498 0.0008392736290310544
Valid MSE: 0.004364873286307094 0.003849422346637954 0.0005154509270805792
	 Valid pr

Test MSE: 0.0004633088513397325 0.0004253548572801706 3.795399309250424e-05
	 Test performance: {'mse': 0.00042524911018399907, 'mrrt': 0.022787745116612922, 'btl': 1.0153988432430197}
epoch: 14 time: 122.4092 
Starting epoch =  15
Train Loss: 0.00048708246145365964 0.00045745686153610663 2.962559967297263e-05
Valid MSE: 0.0005398628824286066 0.0005087461233040177 3.111676079199588e-05
	 Valid preformance: {'mse': 0.9926218166872647, 'mrrt': 0.006820190281870942, 'btl': 253.0}
Test MSE: 0.0004521610557552592 0.0004137766099645941 3.8384444255652966e-05
	 Test performance: {'mse': 0.00041369187854754043, 'mrrt': 0.020658669340219427, 'btl': 0.6967393907252699}
Better valid loss: 0.0005398628824286066
epoch: 15 time: 121.8955 
Starting epoch =  16
Train Loss: 0.0004922039270978358 0.00046466153350617815 2.754239358551245e-05
Valid MSE: 0.0005332876069241181 0.0005054261439779241 2.7861462686338373e-05
	 Valid preformance: {'mse': 1.003297715802389, 'mrrt': 0.0066229096946975674, 'btl': 2

Train Loss: 0.00045846917004186056 0.0004439457174551019 1.4523452612568665e-05
Valid MSE: 0.0005149825529005039 0.0005002458297346334 1.4736723456403451e-05
	 Valid preformance: {'mse': 1.0029112248021805, 'mrrt': 0.005334884664788119, 'btl': 253.0}
Test MSE: 0.00040927728812921525 0.0003912355907370419 1.80416969546949e-05
	 Test performance: {'mse': 0.00039116513012375136, 'mrrt': 0.025858871634852756, 'btl': 1.0365304075239692}
epoch: 31 time: 114.2007 
Starting epoch =  32
Train Loss: 0.00046050815219279834 0.0004461762027121212 1.4331949145761846e-05
Valid MSE: 0.0005497699996109106 0.0005353160280690876 1.4453972638991202e-05
	 Valid preformance: {'mse': 1.0110885524033981, 'mrrt': 0.005910779025512636, 'btl': 253.0}
Test MSE: 0.00044225257249619766 0.00042464984652099006 1.76027258140313e-05
	 Test performance: {'mse': 0.0004246047596113326, 'mrrt': 0.0264008261896617, 'btl': 1.105587910511531}
epoch: 32 time: 112.1601 
Starting epoch =  33
Train Loss: 0.00046815890557293124 0.

	 Test performance: {'mse': 0.0004606149856142965, 'mrrt': 0.031629909679625125, 'btl': 1.5560290497378446}
epoch: 47 time: 106.4793 
Starting epoch =  48
Train Loss: 0.0004524841529436762 0.0004424833470398585 1.0000805842058046e-05
Valid MSE: 0.0005272567648226248 0.000517853901588491 9.402863882870508e-06
	 Valid preformance: {'mse': 1.0083070365975415, 'mrrt': 0.006290658545732941, 'btl': 253.0}
Test MSE: 0.0004127696242462451 0.0004039606756583752 8.808948597463682e-06
	 Test performance: {'mse': 0.00040390569020904186, 'mrrt': 0.03777941155095809, 'btl': 1.3925193942850456}
epoch: 48 time: 110.8550 
Starting epoch =  49
Train Loss: 0.00045708678599849746 0.00044721059270700237 9.876193750236829e-06
Valid MSE: 0.0005282714442901557 0.0005170502447562906 1.1221199770261592e-05
	 Valid preformance: {'mse': 0.9886683008471651, 'mrrt': 0.0059495051770165645, 'btl': 253.0}
Test MSE: 0.0004137732015008044 0.0004029372083917182 1.0835993274100045e-05
	 Test performance: {'mse': 0.0004028

In [13]:
#alpha=1, itr = 4, rel = sector
pred_all = RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.10608068416304131 0.10524141082848498 0.0008392736290310544
Valid MSE: 0.004364873286307094 0.003849422346637954 0.0005154509270805792
	 Valid pr

Test MSE: 0.0004633088513397325 0.0004253548572801706 3.795399309250424e-05
	 Test performance: {'mse': 0.00042524911018399907, 'mrrt': 0.022787745116612922, 'btl': 1.0153988432430197}
epoch: 14 time: 106.1048 
Starting epoch =  15
Train Loss: 0.00048708246145365964 0.00045745686153610663 2.962559967297263e-05
Valid MSE: 0.0005398628824286066 0.0005087461233040177 3.111676079199588e-05
	 Valid preformance: {'mse': 0.9926218166872647, 'mrrt': 0.006820190281870942, 'btl': 253.0}
Test MSE: 0.0004521610557552592 0.0004137766099645941 3.8384444255652966e-05
	 Test performance: {'mse': 0.00041369187854754043, 'mrrt': 0.020658669340219427, 'btl': 0.6967393907252699}
Better valid loss: 0.0005398628824286066
epoch: 15 time: 106.2420 
Starting epoch =  16
Train Loss: 0.0004922039270978358 0.00046466153350617815 2.754239358551245e-05
Valid MSE: 0.0005332876069241181 0.0005054261439779241 2.7861462686338373e-05
	 Valid preformance: {'mse': 1.003297715802389, 'mrrt': 0.0066229096946975674, 'btl': 2

Train Loss: 0.00045846917004186056 0.0004439457174551019 1.4523452612568665e-05
Valid MSE: 0.0005149825529005039 0.0005002458297346334 1.4736723456403451e-05
	 Valid preformance: {'mse': 1.0029112248021805, 'mrrt': 0.005334884664788119, 'btl': 253.0}
Test MSE: 0.00040927728812921525 0.0003912355907370419 1.80416969546949e-05
	 Test performance: {'mse': 0.00039116513012375136, 'mrrt': 0.025858871634852756, 'btl': 1.0365304075239692}
epoch: 31 time: 106.1176 
Starting epoch =  32
Train Loss: 0.00046050815219279834 0.0004461762027121212 1.4331949145761846e-05
Valid MSE: 0.0005497699996109106 0.0005353160280690876 1.4453972638991202e-05
	 Valid preformance: {'mse': 1.0110885524033981, 'mrrt': 0.005910779025512636, 'btl': 253.0}
Test MSE: 0.00044225257249619766 0.00042464984652099006 1.76027258140313e-05
	 Test performance: {'mse': 0.0004246047596113326, 'mrrt': 0.0264008261896617, 'btl': 1.105587910511531}
epoch: 32 time: 106.0854 
Starting epoch =  33
Train Loss: 0.00046815890557293124 0.

	 Test performance: {'mse': 0.0004606149856142965, 'mrrt': 0.031629909679625125, 'btl': 1.5560290497378446}
epoch: 47 time: 105.9637 
Starting epoch =  48
Train Loss: 0.0004524841529436762 0.0004424833470398585 1.0000805842058046e-05
Valid MSE: 0.0005272567648226248 0.000517853901588491 9.402863882870508e-06
	 Valid preformance: {'mse': 1.0083070365975415, 'mrrt': 0.006290658545732941, 'btl': 253.0}
Test MSE: 0.0004127696242462451 0.0004039606756583752 8.808948597463682e-06
	 Test performance: {'mse': 0.00040390569020904186, 'mrrt': 0.03777941155095809, 'btl': 1.3925193942850456}
epoch: 48 time: 106.0285 
Starting epoch =  49
Train Loss: 0.00045708678599849746 0.00044721059270700237 9.876193750236829e-06
Valid MSE: 0.0005282714442901557 0.0005170502447562906 1.1221199770261592e-05
	 Valid preformance: {'mse': 0.9886683008471651, 'mrrt': 0.0059495051770165645, 'btl': 253.0}
Test MSE: 0.0004137732015008044 0.0004029372083917182 1.0835993274100045e-05
	 Test performance: {'mse': 0.0004028

In [ ]:
#alpha=1, itr = 5, rel = sector
pred_all = RR_LSTM.train()

device name: /cpu:0


/tmp/ipykernel_3860/507095565.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_3860/507095565.py:168: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
inner product weight
inner_weight =  (1026, 1026)
weight =  (1026, 1026)
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.10608068747427063 0.10524141414443383 0.0008392736339914876
Valid MSE: 0.004364873522833463 0.0038494225850121844 0.0005154509434803566
	 Valid preformance: {'mse': 

Test MSE: 0.0005105323753052466 0.00045370197015468677 5.683040406070119e-05
	 Test performance: {'mse': 0.00045366521158251324, 'mrrt': 0.0144442129089728, 'btl': 0.7288201435294468, 'btl5': 0.7683657993155064, 'btl10': 0.6433084380343655}
epoch: 12 time: 123.1127 
Starting epoch =  13
Train Loss: 0.0004935354802394097 0.00045971873518578217 3.381674455832163e-05
Valid MSE: 0.0005428634237706853 0.0005076832791336716 3.5180142514859436e-05
	 Valid preformance: {'mse': 0.99459533006578, 'mrrt': 0.005481641378088061, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00047541770151765224 0.0004272581258506754 4.815957552882575e-05
	 Test performance: {'mse': 0.00042717949511041585, 'mrrt': 0.01369986915457887, 'btl': 0.4058374743035529, 'btl5': 0.7226852131949273, 'btl10': 0.6983963530754406}
Better valid loss: 0.0005428634237706853
epoch: 13 time: 124.7045 
Starting epoch =  14
Train Loss: 0.0004886591421971978 0.0004573991136763849 3.126002750807832e-05
Valid MSE: 0.000563235593

Test MSE: 0.0004051222484598117 0.00038668407412896223 1.8438174211886025e-05
	 Test performance: {'mse': 0.0003866162498417793, 'mrrt': 0.03369295906806287, 'btl': 1.2242800382955465, 'btl5': 1.226962163049031, 'btl10': 1.362353644994437}
epoch: 26 time: 132.7738 
Starting epoch =  27
Train Loss: 0.0004706284471958989 0.0004541812820330174 1.6447165143216732e-05
Valid MSE: 0.0005185684106162092 0.0005001361525270142 1.843225860890622e-05
	 Valid preformance: {'mse': 1.0029742676292261, 'mrrt': 0.004383512156419418, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00041104006023000815 0.00039296473716857216 1.8075322071353134e-05
	 Test performance: {'mse': 0.00039290513003863555, 'mrrt': 0.03453290331752005, 'btl': 1.2399819096899591, 'btl5': 1.2906555525842136, 'btl10': 1.3401618523203083}
epoch: 27 time: 130.2680 
Starting epoch =  28
Train Loss: 0.0004725914069559941 0.00045622728603366585 1.6364120409201172e-05
Valid MSE: 0.0005242151139158061 0.0005079490283461627 1.62660

In [8]:
#alpha=1, itr = 2, rel = sector, NASDAQ, implicit
pred_all = RR_LSTM.train()

device name: /cpu:0
#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
#################################
sum weight
head_weight =  (1026, 1)
tail_weight =  (1026, 1)
weight =  (1026, 1026)
#################################
Instructions for updating:
dim is deprecated, use axis instead
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/tmp/ipykernel_3612/1823610647.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_3612/1823610647.py:134: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  head_weight = tf.layers.dense(feature, units=1,
/tmp/ipykernel_3612/1823610647.py:136: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tail_weight = tf.layers.dense(feature, units=1,
/tmp/ipykernel_3612/1823610647.py:168: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################


2023-04-28 14:56:23.687019: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/parth/hadoop-3.2.4/lib/native:
2023-04-28 14:56:23.687080: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-28 14:56:23.687110: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parth-HP-ProBook-430-G6): /proc/driver/nvidia/version does not exist
2023-04-28 14:56:23.689565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 14:56:24.381095: I tensorflow/com

Starting epoch =  0
Train Loss: 0.022771691064652357 0.02217694695030553 0.0005947441267159554
Valid MSE: 0.0013676554000186217 0.001134343740035085 0.00023331166142717905
	 Valid preformance: {'mse': 0.9929000713959802, 'mrrt': 0.00507400129048609, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.0014512972191039017 0.0012541614678680487 0.00019713575249456287
	 Test performance: {'mse': 0.0012543001337172062, 'mrrt': 0.013187531328590301, 'btl': 0.18970057426486164, 'btl5': 0.6608624875487291, 'btl10': 0.9929280112672131}
Better valid loss: 0.0013676554000186217
epoch: 0 time: 154.2595 
Starting epoch =  1
Train Loss: 0.0010777235269464704 0.0009036776714335271 0.00017404585461819707
Valid MSE: 0.000857147215616091 0.0007181411721311244 0.00013900604565043024
	 Valid preformance: {'mse': 0.9930794546187712, 'mrrt': 0.009506548459053547, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.0008019034267906178 0.0006797473857037817 0.0001221560413938385
	 Test performance: 

Train Loss: 0.0004630018556584931 0.0004466695620755451 1.6332293778981006e-05
Valid MSE: 0.000549476398090418 0.0005316292755153318 1.78471224704221e-05
	 Valid preformance: {'mse': 0.9861094035159849, 'mrrt': 0.0057150376569928265, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00043112403824424394 0.0004135981157425995 1.7525921979740278e-05
	 Test performance: {'mse': 0.00041348866283336913, 'mrrt': 0.022052667826313054, 'btl': 0.6696199825382791, 'btl5': 0.8971592852904005, 'btl10': 1.0383673929987691}
epoch: 14 time: 116.8344 
Starting epoch =  15
Train Loss: 0.00046323811675884406 0.0004469425367731035 1.629558054741505e-05
Valid MSE: 0.0005130155619727226 0.0004984646065355581 1.4550956222144975e-05
	 Valid preformance: {'mse': 0.9941388139947664, 'mrrt': 0.012078603186962836, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00039496691406307595 0.00038156135708687015 1.3405555894022182e-05
	 Test performance: {'mse': 0.00038147078206137705, 'mrrt': 0.03139230

	 Valid preformance: {'mse': 0.9900908037094319, 'mrrt': 0.00827905361537304, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.0004136354875169254 0.00039660956784011995 1.7025919277702345e-05
	 Test performance: {'mse': 0.0003965075403266025, 'mrrt': 0.021587430725227782, 'btl': 0.5188319117878564, 'btl5': 0.9420952803629921, 'btl10': 0.9932862491201385}
epoch: 28 time: 117.1779 
Starting epoch =  29
Train Loss: 0.0004626974177144065 0.0004499124440543882 1.2784973563845376e-05
Valid MSE: 0.0005183933182187113 0.0004974547499307572 2.093856938512227e-05
	 Valid preformance: {'mse': 1.0018164900179136, 'mrrt': 0.008609831937419026, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.0004115988152766726 0.00039082409002040103 2.0774724742042524e-05
	 Test performance: {'mse': 0.0003907610433897783, 'mrrt': 0.021918484555732146, 'btl': 0.5894065655884333, 'btl5': 0.9204441684705675, 'btl10': 1.0919688496942408}
epoch: 29 time: 116.7742 
Starting epoch =  30
Train Loss: 0.000

Train Loss: 0.00046928296142141336 0.0004569714165857452 1.2311545511951211e-05
Valid MSE: 0.0005242747488193258 0.0005118199500545401 1.2454798874863454e-05
	 Valid preformance: {'mse': 0.9896558143376314, 'mrrt': 0.00825276352885099, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.0004116589277034688 0.00039892016748164854 1.2738760039537583e-05
	 Test performance: {'mse': 0.0003988144945847768, 'mrrt': 0.020134090168101332, 'btl': 0.558750513533596, 'btl5': 0.9598460659442933, 'btl10': 1.0488487448543329}
epoch: 43 time: 116.5455 
Starting epoch =  44
Train Loss: 0.0004588442298464125 0.00044733736086109764 1.1506868352048391e-05
Valid MSE: 0.0005905627034417916 0.0005815584134700383 9.004289561217442e-06
	 Valid preformance: {'mse': 1.0172149386776699, 'mrrt': 0.008267514855537856, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00047445910705748613 0.0004668842453680508 7.574862740918667e-06
	 Test performance: {'mse': 0.0004668514889659666, 'mrrt': 0.02965502396

In [12]:
#alpha=1, itr = 3, rel = sector, NASDAQ, implicit, seed = 590
pred_all = RR_LSTM.train()

device name: /cpu:0


/tmp/ipykernel_3612/881063778.py:112: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  rel_weight = tf.layers.dense(relation, units=1, activation=leaky_relu)
/tmp/ipykernel_3612/881063778.py:134: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  head_weight = tf.layers.dense(feature, units=1,
/tmp/ipykernel_3612/881063778.py:136: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tail_weight = tf.layers.dense(feature, units=1,
/tmp/ipykernel_3612/881063778.py:168: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  prediction = tf.layers.dense(


#################################
ground_truth =  (1026, 1)
mask =  (1026, 1)
feature =  (1026, 64)
base_price =  (1026, 1)
all_one =  (1026, 1)
relation =  (1026, 1026, 97)
rel_mask =  (1026, 1026)
rel_weight =  (1026, 1026, 1)
#################################
#################################
sum weight
head_weight =  (1026, 1)
tail_weight =  (1026, 1)
weight =  (1026, 1026)
#################################
weight_masked =  (1026, 1026)
outputs_proped =  (1026, 64)
outputs_concated =  (1026, 128)
prediction type =  <class 'tensorflow.python.framework.ops.Tensor'>
prediction =  (1026, 1)
#################################
return_ratio type =  <class 'tensorflow.python.framework.ops.Tensor'>
return_ratio =  (1026, 1)
#################################
reg_loss =  ()
pre_pw_dif =  (1026, 1026)
gt_pw_dif =  (1026, 1026)
rank_loss =  ()
#################################
Starting epoch =  0
Train Loss: 0.022771691129703352 0.022176947026211466 0.0005947441367253133
Valid MSE: 0.00136765549

	 Valid preformance: {'mse': 1.0043567448440422, 'mrrt': 0.011549724705408429, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00040568707926878664 0.00039177555422630664 1.3911524743152653e-05
	 Test performance: {'mse': 0.00039170885713387597, 'mrrt': 0.026962771632614905, 'btl': 1.0126657318905927, 'btl5': 0.9799337633536197, 'btl10': 1.0893079092398688}
epoch: 12 time: 114.9440 
Starting epoch =  13
Train Loss: 0.00046463643784345305 0.00044816518025441295 1.6471258449987423e-05
Valid MSE: 0.0005194792259620348 0.0005015873095313538 1.789191708032001e-05
	 Valid preformance: {'mse': 0.9929024565098409, 'mrrt': 0.005323408793243962, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00040726058737950533 0.00038981830105892844 1.7442285422594796e-05
	 Test performance: {'mse': 0.0003897265923691799, 'mrrt': 0.0187689507022302, 'btl': 0.42448695458006114, 'btl5': 0.7125070251931901, 'btl10': 1.017857970250771}
epoch: 13 time: 114.6670 
Starting epoch =  14
Train Loss: 0

	 Test performance: {'mse': 0.0003882109855656603, 'mrrt': 0.024265536667566725, 'btl': 0.7245820969692431, 'btl5': 0.8685255107644483, 'btl10': 1.1181200644437932}
epoch: 26 time: 114.6820 
Starting epoch =  27
Train Loss: 0.000465517178177047 0.0004524338388315251 1.308333922200266e-05
Valid MSE: 0.00051232680163507 0.0004992352188555598 1.3091582959965481e-05
	 Valid preformance: {'mse': 1.0033036631825574, 'mrrt': 0.008918409841207299, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00039977150726552564 0.0003879635980011864 1.1807908316469262e-05
	 Test performance: {'mse': 0.0003878983452590126, 'mrrt': 0.02552505944887712, 'btl': 0.6414590183994733, 'btl5': 1.0949731878994509, 'btl10': 1.0918179955886447}
epoch: 27 time: 116.7210 
Starting epoch =  28
Train Loss: 0.0004673813141306636 0.0004537539572550953 1.3627356960987035e-05
Valid MSE: 0.000526719530748706 0.0005104314339058917 1.6288097889455023e-05
	 Valid preformance: {'mse': 0.990088765874364, 'mrrt': 0.00827384

Valid MSE: 0.0005122425747455083 0.0004991646377357751 1.3077938206151885e-05
	 Valid preformance: {'mse': 0.993375679750062, 'mrrt': 0.009139485568503245, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00039982891635171567 0.00038669517073182993 1.3133745493247215e-05
	 Test performance: {'mse': 0.00038659833560567735, 'mrrt': 0.020134090168101332, 'btl': 0.558750513533596, 'btl5': 0.9034784523362761, 'btl10': 1.0409696466434983}
epoch: 41 time: 115.1727 
Starting epoch =  42
Train Loss: 0.00046014181930946214 0.00044731429618361684 1.282752386511699e-05
Valid MSE: 0.0005045854018784926 0.0004929738178260676 1.1611582058393984e-05
	 Valid preformance: {'mse': 0.9987796784152236, 'mrrt': 0.01482230199974159, 'btl': 253.0, 'btl5': 253.0, 'btl10': 253.0}
Test MSE: 0.00038953718617813906 0.0003788325647511107 1.0704621037519031e-05
	 Test performance: {'mse': 0.00037875403748509085, 'mrrt': 0.03330199295506882, 'btl': 1.246041600476019, 'btl5': 1.487763689586425, 'btl10': 1.3005